# Manejo de Categorías de Alta Cardinalidad en Bases de Conocimiento SQL para Agentes LLM tipo RAG

Este proyecto utiliza un sistema de embeddings basado en TF-IDF para identificar similitudes de escritura entre variables categóricas de alta cardinalidad en tablas SQL. A continuación, se describe cada paso del proceso.

In [1]:
import sys
import os

sys.path.append(os.path.abspath(".."))

In [2]:
import numpy as np
import uuid

from tqdm import tqdm
from google.cloud import firestore
from google.cloud.firestore_v1.vector import Vector
from langchain_google_firestore import FirestoreVectorStore

from WordMatchSim.tfidf import SimpleTfidfVectorizer
from WordMatchSim.distance_metrics import cosine_similarity
from WordMatchSim.embeddings import CustomTfidfEmbeddings

from dotenv import load_dotenv

load_dotenv()

True

In [3]:
GCP_PROJECT = os.getenv("GCP_PROJECT")
GCP_FIRESTORE_DATABASE = os.getenv("GCP_FIRESTORE_DATABASE")
GCP_FIRESTORE_COLLECTION = os.getenv("GCP_FIRESTORE_COLLECTION")

### Definición de las variables categóricas

En esta sección, se define una lista de marcas (variable categórica) que servirán como ejemplo para generar los vectores.

In [4]:
categorical_words = [
    "Nike", "Adidas", "Puma", "Under Armour", "Reebok", "Asics", "New Balance", "Converse", "Vans", "Fila",
    "Columbia Sportswear", "Patagonia", "The North Face", "Mizuno", "Salomon", "Saucony", "Brooks", "Lululemon",
    "Quiksilver", "Roxy", "Billabong", "O'Neill", "Rip Curl", "DC Shoes", "Oakley", "Helly Hansen", "La Sportiva",
    "Hoka One One", "Merrell", "K-Swiss", "Skechers", "Kappa", "Ellesse", "Diadora", "Umbro", "Speedo",
    "Arena", "Castore", "Dunlop", "Prince", "Wilson", "Head", "Babolat", "Yonex", "Spalding", "Slazenger",
    "Callaway", "Titleist", "Ping", "TaylorMade", "Cobra Golf", "FootJoy", "Ecco", "Garmin", "Polar", "Suunto",
    "Timberland", "Mountain Hardwear", "Arc'teryx", "Gore-Tex", "Hummel", "Joma", "Lotto", "Macron",
    "Mammut", "Odlo", "Ortovox", "Peak Performance", "Rab", "Dynafit", "Icebreaker", "Black Diamond",
    "Carhartt", "Champion", "Everlast", "Le Coq Sportif", "Lonsdale", "Mitre", "Penfield", "Protest",
    "Reef", "RVCA", "Volcom", "Altra", "Bogner", "Descente", "Eleven by Venus", "Errea", "Fila",
    "Gregory", "Huf", "Iffley Road", "Karhu", "Karbon", "Karrimor", "Kjus", "Montbell", "Napapijri",
    "Norrona", "Ortlieb", "Rains", "Samsonite", "Schöffel", "Sea to Summit"
]

### Creación del vectorizador TF-IDF

Se utiliza `SimpleTfidfVectorizer` para crear un vectorizador basado en TF-IDF que genera n-gramas de caracteres. Este enfoque permite capturar similitudes en la escritura de las marcas al descomponer las palabras en secuencias de caracteres. En este caso, se utilizan bigramas (`ngram_range=(1, 2)`) para tokenizar cada palabra, lo cual es adecuado dado la corta longitud de los documentos (nombres de marcas) en el corpus.

In [5]:
vectorizer = SimpleTfidfVectorizer(analyzer='char', ngram_range=(1, 2))
vectorizer.fit(categorical_words)

### Generación de vectores

El vectorizador TF-IDF previamente creado es encapsulado dentro de una clase Embeddings de LangChain. Este paso permite que el vectorizador sea interpretado como un servicio de embeddings dentro del ecosistema de LangChain, facilitando su integración en flujos de trabajo RAG con LLMs.

In [6]:
embeddings = CustomTfidfEmbeddings(vectorizer)

### Cálculo de vectores de las categorías

Se calculan los vectores correspondientes a las categorías (en este caso marcas) utilizando el método de embeddings y se almacenan para futuras comparaciones.

In [7]:
categorical_vectors = embeddings.embed_documents(categorical_words)

### Generación de un vector de consulta

Aquí se toma una palabra de consulta (por ejemplo, 'esquetchers') y se genera su embedding. Este embedding será comparado con los vectores de las categorias (marcas).

In [8]:
query_word = "esquetchers"
query_vector = embeddings.embed_query(query_word)

### Cálculo de la similitud coseno

Se utiliza la similitud coseno para comparar el vector de consulta con los vectores de las palabras categóricas y se identifica la categoría más similar.

In [9]:
similarities = cosine_similarity(query_vector, categorical_vectors)
most_similar_index = np.argmax(similarities)
most_similar_word = categorical_words[most_similar_index]
print(f"La palabra más similar a '{query_word}' es '{most_similar_word}'")

La palabra más similar a 'esquetchers' es 'Skechers'


### Conexión y carga a Firestore

A continuación, se conecta a la base de datos Firestore de Google Cloud y se almacenan los vectores junto con las palabras categóricas en una colección llamada `brands`.

In [10]:
db = firestore.Client(
    project=GCP_PROJECT,
    database=GCP_FIRESTORE_DATABASE
)

for word, vector in tqdm(zip(categorical_words, categorical_vectors)):
    document = {
        "content": word,
        "embedding": Vector(vector)
    }

    db.collection(GCP_FIRESTORE_COLLECTION).document(
        str(uuid.uuid4())).set(document)

### Verificación de las dimensiones de los vectores

Se imprimen las dimensiones de los vectores generados para asegurar que coinciden con lo esperado.

In [15]:
n_documents, vector_dim = np.array(categorical_vectors).shape
print(f'Emdedding dimension: {vector_dim}')

Emdedding dimension: 380


### Creación de índices en Firestore

Se ejecuta un comando para crear índices en Firestore, lo que optimiza las búsquedas vectoriales sobre los vectores almacenados.

```bash
!gcloud firestore indexes composite create --project=GCP_PROJECT --collection-group=GCP_FIRESTORE_COLLECTION --query-scope=COLLECTION --field-config=vector-config='{"dimension":"380","flat": "{}"}',field-path=embedding
```

### Búsqueda por similitud

Finalmente, se utiliza el `FirestoreVectorStore` para realizar una búsqueda por similitud, recuperando los documentos más cercanos en función de los vectores generados.

In [12]:
vector_store = FirestoreVectorStore(
    client=db,
    collection=GCP_FIRESTORE_COLLECTION,
    embedding_service=embeddings
)

In [16]:
relevant_docs = vector_store.similarity_search('esquetchers', 5)

for doc in relevant_docs:
    print(doc.page_content)

Skechers
Descente
Converse
Ellesse
Protest
